In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset = pd.read_json('mergeBefore.json')

In [ ]:
from pickle import APPEND
i=0
for i in range(len(dataset.index)):
    new=dataset.CVE_Items[i]
    if new['impact'] !={}:
        id=new['cve']["CVE_data_meta"]["ID"].lower()
        drc=new['cve']['description']['description_data'][0]['value'].lower()

        accessVector=new['impact']['baseMetricV2']['cvssV2']['accessVector']

        accessComplexity=new['impact']['baseMetricV2']['cvssV2']['accessComplexity']
        impact=new['impact']['baseMetricV2']['obtainAllPrivilege']

        if accessVector=='LOCAL' and accessComplexity=='HIGH':
            needPri="admin/root"
        elif accessVector=='LOCAL' and accessComplexity=='MEDIUM':
            needPri="unknown"
        elif accessVector=='LOCAL' and accessComplexity=='LOW':
            needPri="nonprivileged"
        elif accessVector=='NETWORK' and accessComplexity=='MEDIUM':
            needPri="nonprivileged"
        elif accessVector=='NETWORK' and accessComplexity=='LOW':
            needPri="access"
        elif accessVector=='NETWORK' and accessComplexity=='LOW':
            needPri="access"
        elif accessVector=='NETWORK' and accessComplexity=='HIGH':
            needPri="unknown"
    
        if 'NETWORK' in accessVector:
            actVector='remote'
        else:
            actVector='non-remote'
        
        if  new['impact']['baseMetricV2']['obtainAllPrivilege'] ==True:
            impact="privileged-gained(rce)_admin/root"

        elif    new['impact']['baseMetricV2']['obtainUserPrivilege'] ==True:
            impact="privileged-gained(rce)_nonprivileged"

        elif    new['impact']['baseMetricV2']['obtainOtherPrivilege'] ==True:
            impact="privileged-gained(rce)_unknown"
        elif    new['impact']['baseMetricV2']['cvssV2']['confidentialityImpact']== 'NONE' and new['impact']['baseMetricV2']['cvssV2']['availabilityImpact']=='NONE':
            impact="access"
        elif    new['impact']['baseMetricV2']['cvssV2']['confidentialityImpact']== 'NONE' and new['impact']['baseMetricV2']['cvssV2']['availabilityImpact']=='PARTIAL':
            impact="dos"
        elif    new['impact']['baseMetricV2']['cvssV2']['confidentialityImpact']== 'NONE' and new['impact']['baseMetricV2']['cvssV2']['availabilityImpact']=='COMPLETE':
            impact="dos"  
        elif    new['impact']['baseMetricV2']['cvssV2']['confidentialityImpact']== 'PARTIAL' and new['impact']['baseMetricV2']['cvssV2']['availabilityImpact']=='NONE':
            impact="information-disclosure_other"     
        elif    new['impact']['baseMetricV2']['cvssV2']['confidentialityImpact']== 'PARTIAL' and new['impact']['baseMetricV2']['cvssV2']['availabilityImpact']=='PARTIAL':
            impact="other"  
        elif    new['impact']['baseMetricV2']['cvssV2']['confidentialityImpact']== 'PARTIAL' and new['impact']['baseMetricV2']['cvssV2']['availabilityImpact']=='COMPLETE':
            impact="dos"   
        elif    new['impact']['baseMetricV2']['cvssV2']['confidentialityImpact']== 'COMPLETE' and new['impact']['baseMetricV2']['cvssV2']['availabilityImpact']=='NONE':
             if new['impact']['baseMetricV2']['cvssV2']['integrityImpact']== 'COMPLETE':
                  impact='information-disclosure_local(credit)_admin/root'
             elif   new['impact']['baseMetricV2']['cvssV2']['integrityImpact']== 'PARTIAL':
                  impact="information-disclosure_local(credit)_nonprivileged" 
             elif   new['impact']['baseMetricV2']['cvssV2']['integrityImpact']== 'NONE':
                  impact='information-disclosure_local(credit)_unknown'
        elif    new['impact']['baseMetricV2']['cvssV2']['confidentialityImpact']== 'COMPLETE' and new['impact']['baseMetricV2']['cvssV2']['availabilityImpact']=='NONE':
            if new['impact']['baseMetricV2']['cvssV2']['integrityImpact']== 'COMPLETE':
                  impact='information-disclosure_other-target(credit)_nonprivileged'
            elif   new['impact']['baseMetricV2']['cvssV2']['integrityImpact']== 'PARTIAL':
                  impact="information-disclosure_local(credit)_nonprivileged" 
            elif   new['impact']['baseMetricV2']['cvssV2']['integrityImpact']== 'NONE':
                  impact='information-disclosure_other-target(credit)_unknown'
        elif    new['impact']['baseMetricV2']['cvssV2']['confidentialityImpact']== 'COMPLETE' and new['impact']['baseMetricV2']['cvssV2']['availabilityImpact']=='COMPLETE':
            impact="dos"
        data={"cve-number":id,"description":drc,"privilege-required":needPri,"attack-vector":actVector,"impact":impact}
        jsonData=json.dumps(data)
        with open("merge.json","a")as write_file:
            json.dump(data,write_file)
            write_file.write('\n')
print('ok')